In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split,KFold,GridSearchCV
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.svm import SVC
from sklearn.metrics import log_loss
from sklearn.feature_selection import RFE,chi2,SelectKBest
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,GlobalAveragePooling2D,Input
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight
import xgboost as xgb

In [ ]:
featuers_df_train = pd.read_csv('/content/drive/MyDrive/rice disease/train_sift.csv')

In [ ]:
featuers_df_val = pd.read_csv('/content/drive/MyDrive/rice disease/val_sift_v2.csv')

In [ ]:
test_features_df = pd.read_csv('/content/drive/MyDrive/rice disease/test_sift_v2.csv')

In [ ]:
featuers_df_train.describe()

,3,4,5,6,7,8,9,10,11,12,...,631,632,633,634,635,636,637,638,639,Label
count,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,...,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.000000,4272.00000,4272.000000,4272.000000
mean,15.260768,18.988296,12.308287,10.013343,15.076077,63.948034,29.591058,14.658006,16.586845,24.554775,...,22.481507,20.059925,12.907537,10.142322,17.052200,34.697331,17.664326,11.02809,12.990637,0.594101
std,24.452399,28.180596,22.666915,19.418931,23.812201,53.800155,33.931738,22.284717,24.846529,32.913604,...,28.830399,27.406078,22.163208,19.450430,24.692448,38.371185,25.906378,21.45055,22.344693,0.740533
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,1.000000,15.000000,5.000000,1.000000,1.000000,2.000000,...,3.000000,1.000000,0.000000,0.000000,1.000000,5.000000,1.000000,0.00000,1.000000,0.000000
50%,5.000000,6.000000,3.000000,2.000000,5.000000,47.000000,16.000000,6.000000,6.000000,11.000000,...,11.000000,9.000000,4.000000,2.000000,7.000000,19.000000,7.000000,2.00000,3.000000,0.000000
75%,18.000000,24.250000,13.000000,10.000000,17.000000,115.000000,41.250000,19.000000,21.000000,33.000000,...,31.000000,28.000000,15.000000,11.000000,22.000000,54.000000,23.000000,11.00000,15.000000,1.000000
max,155.000000,147.000000,152.000000,148.000000,143.000000,207.000000,173.000000,141.000000,155.000000,167.000000,...,154.000000,153.000000,151.000000,141.000000,147.000000,173.000000,157.000000,143.00000,151.000000,2.000000


In [ ]:
train,test = train_test_split(featuers_df_train.drop('Image_id',axis=1),test_size=0.2,stratify=featuers_df_train['Label'])

In [ ]:
X_train = np.array(train.drop('Label',axis=1))
y_train = OrdinalEncoder().fit_transform(train['Label'].to_numpy().reshape(-1, 1)).reshape(-1)

In [ ]:
X_val = np.array(test.drop('Label',axis=1))
y_val = OrdinalEncoder().fit_transform(test['Label'].to_numpy().reshape(-1, 1)).reshape(-1)

In [ ]:
scaelr = MinMaxScaler()
X_train = scaelr.fit_transform(X_train)
X_test = scaelr.transform(X_test)

In [ ]:
test_features_df.shape

(1145, 640)

In [ ]:
def reduceFeatures(X,y, model, number_of_features):
    rfe = RFE(estimator=model, n_features_to_select=number_of_features)
    features = rfe.fit(X, y)
    return features.support_

In [ ]:
def reduceFeaturesChi2(X,y,number_of_features):
  bestfeatures = SelectKBest(score_func=chi2, k=number_of_features)
  fit = bestfeatures.fit(X,y)
  dfscores = pd.DataFrame(fit.scores_)
  dfcolumns = pd.DataFrame(list(range(X.shape[1])))
  featureScores = pd.concat([dfcolumns,dfscores],axis=1)
  featureScores.columns = ['Specs','Score']
  return list(featureScores.nlargest(number_of_features,'Score')['Specs'])

In [ ]:
X_val.shape

(534, 640)

In [ ]:
good_features_index = reduceFeaturesChi2(X_train,y_train,600)

In [ ]:
_X_train_ = X_train[:,good_features_index]
_X_val_ = X_val[:,good_features_index]

In [ ]:
_X_val_.shape

(534, 600)

In [ ]:
kf = KFold(10,random_state=42,shuffle=True)

In [ ]:
models = []
for train_index, test_index in kf.split(_X_train_):
   lg = LogisticRegression(C=0.0001)
   X_train0, X_val = _X_train_[train_index], _X_train_[test_index]
   y_train0, y_val = y_train[train_index], y_train[test_index]
   lg.fit(X_train0,y_train0)

   y_on_hot_train = OneHotEncoder(sparse=False).fit_transform(y_train0.reshape((-1,1)))
   y_on_hot_val = OneHotEncoder(sparse=False).fit_transform(y_val.reshape((-1,1)))
   print('training loss: ',log_loss(y_on_hot_train,lg.predict_proba(X_train0)),' , validation loss : ',log_loss(y_on_hot_val,lg.predict_proba(X_val)))
   models.append(lg)

In [ ]:
models = []
for train_index, test_index in kf.split(_X_train_):
   svc = SVC(probability=True,class_weight='balanced',C=0.1,gamma='auto')
   X_train0, X_val0 = _X_train_[train_index], _X_train_[test_index]
   y_train0, y_val0 = y_train[train_index], y_train[test_index]
   svc.fit(X_train0,y_train0)

   y_on_hot_train = OneHotEncoder().fit_transform(y_train0.reshape((-1,1)))
   y_on_hot_val = OneHotEncoder().fit_transform(y_val0.reshape((-1,1)))
   print('training loss: ',log_loss(y_on_hot_train,svc.predict_proba(X_train0)),' , validation loss : ',log_loss(y_on_hot_val,svc.predict_proba(X_val0)))
   models.append(svc)

training loss:  0.972465659342974  , validation loss :  1.0024529593942018
training loss:  0.9716018328440787  , validation loss :  0.9974316751733203
training loss:  0.9834552515884337  , validation loss :  0.9236720624405385
training loss:  0.9775066490360929  , validation loss :  0.9690928818089064
training loss:  0.9785619809227343  , validation loss :  0.9584982424128664
training loss:  0.9750032672313135  , validation loss :  0.9906686878330709
training loss:  0.9770689954107294  , validation loss :  0.9794128507151887
training loss:  0.9798733419859273  , validation loss :  0.9514569145589794
training loss:  0.9760747698505596  , validation loss :  0.9929312287479545
training loss:  0.9795646747591453  , validation loss :  0.9554724814329995


In [ ]:
models = []
for train_index, test_index in kf.split(_X_train_):
   rfc = RandomForestClassifier(20,max_depth=15,class_weight='balanced')
   X_train0, X_val0 = _X_train_[train_index], _X_train_[test_index]
   y_train0, y_val0 = y_train[train_index], y_train[test_index]
   rfc.fit(X_train0,y_train0)

   y_on_hot_train = OneHotEncoder().fit_transform(y_train0.reshape((-1,1)))
   y_on_hot_val = OneHotEncoder().fit_transform(y_val0.reshape((-1,1)))
   print('training loss: ',log_loss(y_on_hot_train,rfc.predict_proba(X_train0)),' , validation loss : ',log_loss(y_on_hot_val,rfc.predict_proba(X_val0)))
   models.append(rfc)

training loss:  0.31324913414099537  , validation loss :  0.9710861077840393
training loss:  0.3028788024155559  , validation loss :  0.991953121250737
training loss:  0.3068259132577469  , validation loss :  0.9140541744415167
training loss:  0.3083367592240404  , validation loss :  0.9628503383299573
training loss:  0.31142844897458477  , validation loss :  0.9668950482919094
training loss:  0.3164214528956351  , validation loss :  0.9730585337586295
training loss:  0.30365415041134014  , validation loss :  0.9851052183644589
training loss:  0.2952179568195889  , validation loss :  0.955640384681014
training loss:  0.29968835814578965  , validation loss :  0.9946867635129675
training loss:  0.30994765394497176  , validation loss :  0.9470723694112766


In [ ]:
model = xgb.XGBClassifier(learning_rate=0.1,n_estimators=20,objective='multi:softmax',reg_lambda=5)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_one_hot_train = OneHotEncoder().fit_transform(y_train.reshape((-1,1)))
y_one_hot_val = OneHotEncoder().fit_transform(y_val.reshape((-1,1)))

In [ ]:
y_one_hot_val.shape

(534, 3)

In [ ]:
y_val

In [ ]:
model.fit(_X_train_,y_train)

In [ ]:
print('training loss: ',log_loss(y_one_hot_train,model.predict_proba(X_train)),' , validation loss : ',log_loss(y_one_hot_val,model.predict_proba(X_val)))

training loss:  0.8550764535054955  , validation loss :  0.9568253762266609


In [ ]:
gsc.fit(_X_train_,y_train)

In [ ]:
y_on_hot_val.shape

(534, 3)

In [ ]:
log_loss(y_on_hot_val,gsc.best_estimator_.predict_proba(_X_val_))

1.0603868846316182

In [ ]:
test = pd.read_csv('/content/SampleSubmission.csv')

In [ ]:
test_featurs = np.array(test_features_df)

In [ ]:
test_featurs = test_featurs[:,good_features_index]

In [ ]:
test_pred = models[6].predict_proba(test_featurs)

In [ ]:
test.loc[:,['blast','brown','healthy']] = test_pred
test.to_csv('sub13.csv',index=False)